In [31]:
import pandas as pd
from preProccesPuzzle import PreProcess
from clue_classifier import ClueClassifier
from constraints import Constraint, IdentityConstrain, NextToConstrain, DistanceConstrain,RightConstrain, LeftConstrain, DirectRightConstrain, DirectLeftConstrain, PositionAbsoluteConstrain, PositionAbsoluteNegativeConstrain
from constraint_solver import ConstraintSolver
from concurrent.futures import ProcessPoolExecutor


In [32]:
def constraint_factory(attrs, clues):
    constrains: list[Constraint] = []
    classifier = ClueClassifier()
    for c in clues:
        clue, clue_type = classifier.classify(c)

        if clue_type == "IDENTITY":
            constrains.append(IdentityConstrain(attrs, clue))
        if clue_type == "NEXT_TO":
            constrains.append(NextToConstrain(attrs, clue))
        if clue_type == "LEFT":
            constrains.append(LeftConstrain(attrs, clue))
        if clue_type == "RIGHT":
            constrains.append(RightConstrain(attrs, clue))
        if clue_type == "DISTANCE":
            constrains.append(DistanceConstrain(attrs, clue))
        if clue_type == "DIRECT_LEFT":
            constrains.append(DirectLeftConstrain(attrs, clue))
        if clue_type == "DIRECT_RIGHT":
            constrains.append(DirectRightConstrain(attrs, clue))
        if clue_type == "POSITION_ABSOLUTE":
            constrains.append(PositionAbsoluteConstrain(attrs, clue))
        if clue_type == "POSITION_ABSOLUTE_NEGATIVE":
            constrains.append(PositionAbsoluteNegativeConstrain(attrs, clue))
        if clue_type == "UNKNOWN":
            raise TypeError
    
    return constrains

In [33]:
gridmode = pd.read_parquet("Gridmode-00000-of-00001.parquet")
mc = pd.read_parquet("mc-00000-of-00001.parquet")
ppp = PreProcess()

In [34]:
puzzle = gridmode.puzzle.iloc[388].lower()

attrs, clues = ppp.proccess(puzzle)

print(puzzle)

there are 2 houses, numbered 1 to 2 from left to right, as seen from across the street. each house is occupied by a different person. each house has a unique attribute for each of the following characteristics:
 - each person has a unique name: `arnold`, `eric`
 - each person has a unique type of pet: `cat`, `dog`
 - each person has a unique favorite drink: `tea`, `water`
 - people have unique favorite music genres: `rock`, `pop`
 - the people keep unique animals: `horse`, `cat`
 - people own unique car models: `ford f150`, `tesla model 3`

## clues:
1. the tea drinker is the person who owns a tesla model 3.
2. the person who owns a dog is the cat lover.
3. arnold is the tea drinker.
4. the person who loves pop music is directly left of the person who has a cat.
5. the person who loves rock music is the tea drinker.



In [35]:
constrains: list[Constraint] = constraint_factory(attrs, clues)

Cs = ConstraintSolver(attrs, constrains)

for c in constrains:
    print(c.get_info())

solution = Cs.solve()

Cs.print_solution(solution)

IdentityConstrain:  the tea drinker is the person who owns a tesla model 3.
attr1:('tea', 'drink')
attr2:('tesla model 3', 'car')
attributes:{'name': ['arnold', 'eric'], 'pet': ['cat', 'dog'], 'drink': ['tea', 'water'], 'music': ['rock', 'pop'], 'animals': ['horse', 'cat'], 'car': ['ford f150', 'tesla model 3']}

IdentityConstrain:  the person who owns a dog is the cat lover.
attr1:('dog', 'pet')
attr2:('cat', 'pet')
attributes:{'name': ['arnold', 'eric'], 'pet': ['cat', 'dog'], 'drink': ['tea', 'water'], 'music': ['rock', 'pop'], 'animals': ['horse', 'cat'], 'car': ['ford f150', 'tesla model 3']}

IdentityConstrain:  arnold is the tea drinker.
attr1:('arnold', 'name')
attr2:('tea', 'drink')
attributes:{'name': ['arnold', 'eric'], 'pet': ['cat', 'dog'], 'drink': ['tea', 'water'], 'music': ['rock', 'pop'], 'animals': ['horse', 'cat'], 'car': ['ford f150', 'tesla model 3']}

DirectLeftConstrain: the person who loves pop music is directly left of the person who has a cat.
attr1:('pop', 'm